# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#import scipy.misc
#from scipy.misc import imread, imresize

from PIL import Image
import datetime
import os
from skimage.transform import resize
from imageio import imread

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

#### Lets define the number of frame, image width and image height 

In [4]:
#number of frames
x = 30 

#image width
y = 120 

#image height
z = 120 

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    #create a list of image numbers you want to use for a particular video
    img_idx =  [x for x in range(0,x)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    out_temp = resize(image,(120,120))
                    out_temp = out_temp/127.5-1 #Normalize data
                    
                    #normalise and feed in the image
                    batch_data[folder,idx,:,:,0] = (out_temp[:,:,0]) 
                    batch_data[folder,idx,:,:,1] = (out_temp[:,:,1])                    
                    batch_data[folder,idx,:,:,2] = (out_temp[:,:,2])
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    out_temp = resize(image,(120,120))
                    out_temp = out_temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (out_temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (out_temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (out_temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/Project_data/train'
val_path = '/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# choose the number of epochs
num_epochs = 10

print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

### 1-> Lets use Sequential model for the linear stack of layers.

In [8]:
model_seq_1 = Sequential()

##### Number of filters - 8

In [9]:
model_seq_1.add(Conv3D(8,
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_seq_1.add(BatchNormalization())
model_seq_1.add(Activation('relu'))

model_seq_1.add(MaxPooling3D(pool_size=(2,2,2)))

##### Number of filters - 16

In [10]:
model_seq_1.add(Conv3D(16, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_seq_1.add(BatchNormalization())
model_seq_1.add(Activation('relu'))

model_seq_1.add(MaxPooling3D(pool_size=(2,2,2)))

##### Number of filters - 32

In [11]:
model_seq_1.add(Conv3D(32, 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_seq_1.add(BatchNormalization())
model_seq_1.add(Activation('relu'))

model_seq_1.add(MaxPooling3D(pool_size=(2,2,2)))


##### Number of filters - 64

In [12]:
model_seq_1.add(Conv3D(64,
                 kernel_size=(1,3,3), 
                 padding='same'))
model_seq_1.add(BatchNormalization())
model_seq_1.add(Activation('relu'))

model_seq_1.add(MaxPooling3D(pool_size=(2,2,2)))

In [13]:
#Flatten Layers
model_seq_1.add(Flatten())

model_seq_1.add(Dense(1000, activation='relu'))
model_seq_1.add(Dropout(0.5))

model_seq_1.add(Dense(500, activation='relu'))
model_seq_1.add(Dropout(0.5))

##### Lets use activation function to normalize the output to a probability

In [14]:
#softmax layer
model_seq_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

##### Lets use Adam optimizer as we wants to train the neural network in less time and more efficiently 

In [15]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_seq_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_seq_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
model_seq_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 10


C:\Users\acer\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
67/67 [==============================] - 822s 12s/step - loss: 6.7297 - categorical_accuracy: 0.2044 - val_loss: 1.4087 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-5.15875-0.23680-1.40866-0.36000.h5
Epoch 2/10
67/67 [==============================] - 233s 3s/step - loss: 1.9216 - categorical_accuracy: 0.3585 - val_loss: 1.7125 - val_categorical_accuracy: 0.2900

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-2.41348-0.32338-1.71245-0.29000.h5
Epoch 3/10
67/67 [==============================] - 231s 3s/step - loss: 2.1091 - categorical_accuracy: 0.3167 - val_loss: 1.3871 - val_categorical_accuracy: 0.4300

Epoch 00003: saving model to model_init_2021-04-0500_27_47.853265\model-00003-2.07445-0.24378-1.38708-0.43000.h5
Epoch 4/10
67/67 [==============================] - 253s 4s/step - loss: 1.6167 - categorical_accuracy: 0.2953 - val_loss: 1.4746 - val_categorical_accuracy: 0.32

#### Lets make some variations in batch size
#### Batch Size - 20

In [20]:
train_generator = generator(train_path, train_doc, 20)
val_generator = generator(val_path, val_doc, 20)

In [21]:
model_seq_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 20
Epoch 1/10
67/67 [==============================] - 729s 11s/step - loss: 1.0419 - categorical_accuracy: 0.5787 - val_loss: 0.9910 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-1.04194-0.57874-0.99098-0.65000.h5
Epoch 2/10
67/67 [==============================] - 279s 4s/step - loss: 1.2975 - categorical_accuracy: 0.4876 - val_loss: 1.1466 - val_categorical_accuracy: 0.4900

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-1.29754-0.48756-1.14660-0.49000.h5
Epoch 3/10
67/67 [==============================] - 293s 4s/step - loss: 1.1669 - categorical_accuracy: 0.5622 - val_loss: 1.0060 - val_categorical_accuracy: 0.5900

Epoch 00003: saving model to model_init_2021-04-0500_27_47.853265\model-00003-1.16688-0.56219-1.00602-0.59000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
67/67 [==========

#### Batch Size -  30

In [22]:
train_generator = generator(train_path, train_doc, 30)
val_generator = generator(val_path, val_doc, 30)

In [23]:
model_seq_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 30
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.5950 - categorical_accuracy: 0.7874 Source path =  /Project_data/val ; batch size = 30
Batch:  4 Index: 30
67/67 [==============================] - 767s 11s/step - loss: 0.5950 - categorical_accuracy: 0.7874 - val_loss: 0.6531 - val_categorical_accuracy: 0.6938

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-0.59496-0.78742-0.65306-0.69375.h5
Epoch 2/10
67/67 [==============================] - 246s 4s/step - loss: 0.6111 - categorical_accuracy: 0.8010 - val_loss: 0.6282 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-0.61110-0.80100-0.62815-0.76000.h5
Epoch 3/10
67/67 [==============================] - 255s 4s/step - loss: 0.7064 - categorical_accuracy: 0.7413 - val_loss: 0.7934 - val_categorical_accuracy: 0.7000

Epoch 00003: saving model to model_init_2021-04-0500_27_

#### Batch Size - 40

In [24]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)

In [25]:
model_seq_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.4737 - categorical_accuracy: 0.8296 Source path =  /Project_data/val ; batch size = 40
Batch:  3 Index: 40
67/67 [==============================] - 1550s 23s/step - loss: 0.4737 - categorical_accuracy: 0.8296 - val_loss: 0.6090 - val_categorical_accuracy: 0.7292

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-0.47371-0.82957-0.60903-0.72917.h5
Epoch 2/10
67/67 [==============================] - 1072s 16s/step - loss: 0.4080 - categorical_accuracy: 0.8609 - val_loss: 0.6635 - val_categorical_accuracy: 0.6900

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-0.40795-0.86094-0.66351-0.69000.h5
Epoch 3/10
67/67 [==============================] - 1040s 16s/step - loss: 0.4753 - categorical_accuracy: 0.8227 - val_loss: 0.5912 - val_categorical_accuracy: 0.7300

Epoch 00003: saving model to model_init_2021-04-050

#### Lets Change Optimizer to Adadelta Experiment

In [27]:
import keras
model_seq_1.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_seq_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

In [28]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)

In [29]:
model_seq_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.4196 - categorical_accuracy: 0.8466 Source path =  /Project_data/val ; batch size = 40
Batch:  3 Index: 40
67/67 [==============================] - 1535s 23s/step - loss: 0.4193 - categorical_accuracy: 0.8467 - val_loss: 0.5848 - val_categorical_accuracy: 0.7292

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-0.40436-0.85043-0.58475-0.72917.h5
Epoch 2/10
67/67 [==============================] - 1067s 16s/step - loss: 0.3807 - categorical_accuracy: 0.8701 - val_loss: 0.5653 - val_categorical_accuracy: 0.7150

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-0.36850-0.87210-0.56526-0.71500.h5
Epoch 3/10
67/67 [==============================] - 1041s 16s/step - loss: 0.4315 - categorical_accuracy: 0.8218 - val_loss: 0.5970 - val_categorical_accuracy: 0.7050

Epoch 00003: saving model to model_init_2021-04-050

#### Lets change epochs to 20

In [30]:
num_epochs = 20
print ('# epochs =', num_epochs)

# epochs = 20


In [31]:
import keras
model_seq_1.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_seq_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

In [32]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)

In [33]:
model_seq_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 0.3806 - categorical_accuracy: 0.8674 Source path =  /Project_data/val ; batch size = 40
Batch:  3 Index: 40
67/67 [==============================] - 2440s 36s/step - loss: 0.3807 - categorical_accuracy: 0.8674 - val_loss: 0.5855 - val_categorical_accuracy: 0.7333

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-0.38451-0.86261-0.58548-0.73333.h5
Epoch 2/20
67/67 [==============================] - 1083s 16s/step - loss: 0.4133 - categorical_accuracy: 0.8334 - val_loss: 0.6214 - val_categorical_accuracy: 0.6850

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-0.41126-0.84464-0.62139-0.68500.h5
Epoch 3/20
67/67 [==============================] - 1055s 16s/step - loss: 0.3880 - categorical_accuracy: 0.8471 - val_loss: 0.5894 - val_categorical_accuracy: 0.7100

Epoch 00003: saving model to model_init_2021-04-050

##### The Final Model

In [35]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_last = Sequential()

In [36]:
#1-------------
model_last.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_last.add(BatchNormalization())
model_last.add(Activation('relu'))

model_last.add(MaxPooling3D(pool_size=(2,2,2)))

#2--------------
model_last.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_last.add(BatchNormalization())
model_last.add(Activation('relu'))

model_last.add(MaxPooling3D(pool_size=(2,2,2)))

#3--------------
model_last.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_last.add(BatchNormalization())
model_last.add(Activation('relu'))

model_last.add(MaxPooling3D(pool_size=(2,2,2)))

#4--------------
model_last.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_last.add(Activation('relu'))
model_last.add(Dropout(0.25))

model_last.add(MaxPooling3D(pool_size=(2,2,2)))

#### Lets Flatten the layers

In [37]:
model_last.add(Flatten())

model_last.add(Dense(nb_dense[0], activation='relu'))
model_last.add(Dropout(0.5))

model_last.add(Dense(nb_dense[1], activation='relu'))
model_last.add(Dropout(0.5))

#### Add Softmax Layer

In [38]:
model_last.add(Dense(nb_dense[2], activation='softmax'))

#### Lets optimize using Adam

In [39]:
optimiser = keras.optimizers.Adam() 
model_last.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_last.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_4 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 16)   

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
batch_size = 10
num_epochs = 20
model_last.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 625s 9s/step - loss: 2.3361 - categorical_accuracy: 0.2242 - val_loss: 1.5560 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2021-04-0500_27_47.853265\model-00001-1.85885-0.26395-1.55602-0.41000.h5
Epoch 2/20
67/67 [==============================] - 256s 4s/step - loss: 1.5779 - categorical_accuracy: 0.2979 - val_loss: 1.6063 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2021-04-0500_27_47.853265\model-00002-1.58723-0.27363-1.60630-0.21000.h5
Epoch 3/20
67/67 [==============================] - 260s 4s/step - loss: 1.6518 - categorical_accuracy: 0.2005 - val_loss: 1.5955 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2021-04-0500_27_47.853265\model-00003-1.62376-0.21393-1.59554-0.23000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/20
67/67 [===========